In [53]:
from bs4 import BeautifulSoup
import requests

from time import sleep
import pandas as pd

In [74]:
all_links = []

In [ ]:
soups = []

for n in range(1,295):
    r  = requests.get("http://www.medias-presse.info/category/politique/page/{}/".format(n))
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    
    soups.append(soup)
    sleep(0.2)

In [118]:
for soup in soups:
    ls = []

    for link in soup.find_all('a'):
        url = link.get('href')
        x = url.split("/")
        if len(x) > 3:
            last_value = x[-2]
            last_value2 = x[-1]
            
            if last_value.isdigit():
                last_value = int(last_value)
                if last_value > 0 and last_value < 90000 and last_value2 != "#comments":
                    ls.append(url)
    
    all_links.extend(ls)
    print("Page {} done.".format(n))

all_links = set(all_links)
all_links = list(all_links)

NameError: name 'soups' is not defined

In [153]:
di = {"id":[], "agency": [], "url": [], "title": [], "text":[], "author":[]}

for link in all_links[:1000]:
    sleep(0.1)
    r  = requests.get(link)
    data = r.text
    soup = BeautifulSoup(data, "lxml")

    paragraphs_ls = []

    for paragraph in soup.find_all('p', style="text-align: justify;"):
        paragraphs_ls.append(paragraph.text)

    paragraphs = "\n".join(paragraphs_ls)
    
    _id = link.split("/")[-2]
    author = soup.find('a', rel='author').text
    title = soup.title.text
    
    if paragraphs != "":
        di["id"].append(_id)
        di["agency"].append("medias-presse.info")
        di["url"].append(link)
        di["title"].append(title)
        di["text"].append(paragraphs)
        di["author"].append(author)
    
        print("Article {} completed.".format(_id))

Article 30671 completed.
Article 829 completed.
Article 74583 completed.
Article 64394 completed.
Article 77405 completed.
Article 56114 completed.
Article 28680 completed.
Article 38171 completed.
Article 3180 completed.
Article 48248 completed.
Article 31776 completed.
Article 1217 completed.
Article 68938 completed.
Article 87928 completed.
Article 2994 completed.
Article 5944 completed.
Article 75838 completed.
Article 35246 completed.
Article 24221 completed.
Article 62368 completed.
Article 71 completed.
Article 52594 completed.
Article 32937 completed.
Article 56835 completed.
Article 18057 completed.
Article 70376 completed.
Article 13408 completed.
Article 88734 completed.
Article 65790 completed.
Article 70964 completed.
Article 31822 completed.
Article 89147 completed.
Article 69965 completed.
Article 26704 completed.
Article 42136 completed.
Article 62821 completed.
Article 88179 completed.
Article 179 completed.
Article 2535 completed.
Article 136 completed.
Article 4784 c

Article 76458 completed.
Article 58091 completed.
Article 6900 completed.
Article 29304 completed.
Article 30 completed.
Article 88871 completed.
Article 83478 completed.
Article 41583 completed.
Article 63834 completed.
Article 63717 completed.
Article 4302 completed.
Article 64551 completed.
Article 52662 completed.
Article 62719 completed.
Article 33605 completed.
Article 89105 completed.
Article 26329 completed.
Article 10623 completed.
Article 10543 completed.
Article 44332 completed.
Article 25056 completed.
Article 234 completed.
Article 76191 completed.
Article 711 completed.
Article 72632 completed.
Article 71632 completed.
Article 66875 completed.
Article 76310 completed.
Article 62919 completed.
Article 73079 completed.
Article 24158 completed.
Article 43442 completed.
Article 60125 completed.
Article 49752 completed.
Article 39563 completed.
Article 5966 completed.
Article 34885 completed.
Article 29801 completed.
Article 7664 completed.
Article 23298 completed.
Article 710

Article 194 completed.
Article 66036 completed.
Article 23145 completed.
Article 67742 completed.
Article 50628 completed.
Article 24026 completed.
Article 73695 completed.
Article 65695 completed.
Article 45409 completed.
Article 23967 completed.
Article 867 completed.
Article 87526 completed.
Article 157 completed.
Article 12 completed.
Article 76207 completed.
Article 34637 completed.
Article 65178 completed.
Article 71913 completed.
Article 61683 completed.
Article 3777 completed.
Article 71627 completed.
Article 51148 completed.
Article 270 completed.
Article 68136 completed.
Article 68151 completed.
Article 118 completed.
Article 8243 completed.
Article 7126 completed.
Article 74167 completed.
Article 3502 completed.
Article 76546 completed.
Article 73973 completed.
Article 36623 completed.
Article 75078 completed.
Article 52994 completed.
Article 62260 completed.
Article 47335 completed.
Article 81482 completed.
Article 69894 completed.
Article 69615 completed.
Article 34995 com

Article 83073 completed.
Article 46895 completed.


In [154]:
# Save the model
pd.DataFrame.from_dict(di).to_csv("fakenews.csv", index=False)

In [161]:
di_big = {"id":[], "agency": [], "url": [], "title": [], "text":[], "author":[]}
counter = 0

for link in all_links:
    sleep(0.1)
    r  = requests.get(link)
    data = r.text
    soup = BeautifulSoup(data, "lxml")

    paragraphs_ls = []

    for paragraph in soup.find_all('p', style="text-align: justify;"):
        paragraphs_ls.append(paragraph.text)

    paragraphs = "\n".join(paragraphs_ls)
    
    _id = link.split("/")[-2]
    author = soup.find('a', rel='author').text
    title = soup.title.text
    
    if paragraphs != "":
        di_big["id"].append(_id)
        di_big["agency"].append("media-presse.info")
        di_big["url"].append(link)
        di_big["title"].append(title)
        di_big["text"].append(paragraphs)
        di_big["author"].append(author)

In [162]:
pd.DataFrame.from_dict(di_big).to_csv("fakenews_big.csv", index=False)